In [ ]:
#imports

In [ ]:
#find files

In [ ]:
#load files into pandas data frame

In [ ]:
#filter data

In [ ]:
#calculate noise floor
#determine threshold

# ONLY signal is present data as input for find peaks 

In [ ]:
#find peaks

#select peaks of interest

In [ ]:
#peak prominencces to detect node to peak (amplitude)
#find node latency